In [10]:
# puzzle_generator.py
import random
from typing import Tuple

def generate_puzzle(difficulty: str) -> Tuple[str, int]:
    """
    Generate a math puzzle and return (question_str, correct_answer)
    difficulty in {'easy', 'medium', 'hard'}
    """
    if difficulty == "easy":
        a = random.randint(1, 10)
        b = random.randint(1, 10)
        ops = ['+','-','*','/']
    elif difficulty == "medium":
        a = random.randint(10, 50)
        b = random.randint(1, 20)
        ops = ['+','-','*','/']
    else:  # hard
        a = random.randint(50, 200)
        b = random.randint(10, 50)
        ops = ['+','-','*','/']

    op = random.choice(ops)
    if op == '+':
        ans = a + b
    elif op == '-':
        ans = a - b
    elif op == '/':
        ans = int(a / b)
    else:
        ans = a * b

    question = f"{a} {op} {b}"
    return question, ans


In [11]:
# tracker.py
from typing import List, Dict

# Global simple in-memory attempts list (list of dicts)
attempts: List[Dict] = []

def log_attempt(question: str, user_ans: int, correct_ans: int, time_taken: float) -> None:
    attempts.append({
        'question': question,
        'user_ans': user_ans,
        'correct_ans': correct_ans,
        'correct': (user_ans == correct_ans),
        'time': time_taken
    })

def clear_attempts() -> None:
    attempts.clear()

def get_summary() -> Dict:
    if not attempts:
        return {'total': 0, 'correct': 0, 'accuracy': 0.0, 'avg_time': 0.0}
    total = len(attempts)
    correct = sum(1 for a in attempts if a['correct'])
    avg_time = sum(a['time'] for a in attempts) / total
    return {'total': total, 'correct': correct, 'accuracy': correct / total, 'avg_time': avg_time}

def recent_history(n: int = 10):
    """Return last n attempts (most recent last)."""
    return attempts[-n:]
events = []  # list of events like {'type':'level_change','from':'easy','to':'medium','reason':'3_correct','time':...}

def log_event(event_type: str, meta: dict):
    import time
    events.append({'type': event_type, 'meta': meta, 'ts': time.time()})

def get_events():
    return list(events)



In [12]:
# adaptive_engine.py
from typing import List, Dict

LEVELS = ['easy', 'medium', 'hard']

def next_level(history: List[Dict], current_level: str) -> str:
    """
    Simple rule-based adaptation:
    - If last 3 attempts exist and are all correct -> increase difficulty (if possible)
    - If last 2 attempts exist and are both incorrect -> decrease difficulty (if possible)
    - Else keep same level.
    history: list of attempts (each attempt has 'correct' key). Most recent attempts are at the end.
    """
    if current_level not in LEVELS:
        current_level = 'easy'
    idx = LEVELS.index(current_level)

    # Look at last 3
    if len(history) >= 3:
        last3 = history[-3:]
        if all(a.get('correct', False) for a in last3) and idx < len(LEVELS)-1:
            return LEVELS[idx+1]

    # Look at last 2
    if len(history) >= 2:
        last2 = history[-2:]
        if all(not a.get('correct', False) for a in last2) and idx > 0:
            return LEVELS[idx-1]

    return current_level


In [13]:
# main.py
import time
#from puzzle_generator import generate_puzzle
#from tracker import log_attempt, get_summary, recent_history, clear_attempts
#from adaptive_engine import next_level

def choose_initial_level() -> str:
    print("Choose initial difficulty:")
    print("1) Easy\n2) Medium\n3) Hard")
    pick = input("Enter 1/2/3 (default 1): ").strip()
    return {'1': 'easy', '2': 'medium', '3': 'hard'}.get(pick, 'easy')

def run_session(num_questions: int = 10):
    clear_attempts()

    name = input("Enter learner's name: ").strip() or "Learner"
    level = choose_initial_level()

    print(f"\nHello {name}! Starting at '{level}' level.")
    print(f"You will get {num_questions} questions.\n")

    for qnum in range(1, num_questions + 1):
        question, correct_ans = generate_puzzle(level)

        print(f"Q{qnum} (level = {level})")
        print(f"{question} = ?")

        start_time = time.time()
        user_input = input("Your answer: ").strip()
        time_taken = time.time() - start_time

        try:
            user_ans = int(user_input)
        except ValueError:
            user_ans = None

        is_correct = (user_ans == correct_ans)

        # Log the attempt
        log_attempt(
            question=question,
            user_ans=user_ans if user_ans is not None else -1,
            correct_ans=correct_ans,
            time_taken=time_taken
        )

        # Feedback to user
        if is_correct:
            print("Correct!")
        else:
            print(f"Incorrect. Correct answer is {correct_ans}")

        print(f"⏱ Time taken: {time_taken:.2f} seconds")

        # Decide next difficulty
        history = recent_history(10)
        new_level = next_level(history, level)

        if new_level != level:
            print(f"⬆ Difficulty changed: {level} → {new_level}")
        else:
            print(f"➡ Difficulty remains: {level}")

        level = new_level
        print("-" * 40)

    # Session summary
    summary = get_summary()
    print("\n SESSION SUMMARY")
    print(f"Total Questions: {summary['total']}")
    print(f"Correct Answers: {summary['correct']}")
    print(f"Accuracy: {summary['accuracy'] * 100:.1f}%")
    print(f"Average Time per Question: {summary['avg_time']:.2f} seconds")

if __name__ == "__main__":
    run_session(num_questions=10)


Enter learner's name: nnnn
Choose initial difficulty:
1) Easy
2) Medium
3) Hard
Enter 1/2/3 (default 1): 1

Hello nnnn! Starting at 'easy' level.
You will get 10 questions.

Q1 (level = easy)
9 + 6 = ?
Your answer: 15
✅ Correct!
⏱ Time taken: 8.15 seconds
➡ Difficulty remains: easy
----------------------------------------
Q2 (level = easy)
3 - 8 = ?
Your answer: -5
✅ Correct!
⏱ Time taken: 6.34 seconds
➡ Difficulty remains: easy
----------------------------------------
Q3 (level = easy)
8 - 7 = ?
Your answer: 1
✅ Correct!
⏱ Time taken: 3.44 seconds
⬆ Difficulty changed: easy → medium
----------------------------------------
Q4 (level = medium)
49 + 16 = ?
Your answer: 64
❌ Incorrect. Correct answer is 65
⏱ Time taken: 6.03 seconds
➡ Difficulty remains: medium
----------------------------------------
Q5 (level = medium)
44 + 10 = ?
Your answer: 54
✅ Correct!
⏱ Time taken: 5.91 seconds
➡ Difficulty remains: medium
----------------------------------------
Q6 (level = medium)
33 / 13 = ?
Y